# Configure Scan & System Health Displays Test

## Table of Contents
#### [1 - Global Configuration](#1-global-configuration)
#### [2 - Telescope Setup](#2-test-equipment-setup)
#### [3 - Observation Definition](#3-configure-scan-tests)
#### [4 - Configure System for Scan Commissioning Test ](#4-configure-for-scan)
#### [5 - Hyperlinks and SUT config](#5-hyperlinks-and-sut-config)

## 1 Global Configuration

### 1.1 Import dependencies
##### Importing all libraries that will be used throughout this notebook

In [4]:
import sys

sys.path.append("../../src")

import json
import logging
import os
import pathlib
import time
import tango
from typing import List

import ska_ser_logging
from bokeh.io import output_notebook
from ska_oso_pdm.entities.common.target import (
    CrossScanParameters,
    FivePointParameters,
    RasterParameters,
    SinglePointParameters,
    StarRasterParameters,
)
from ska_oso_pdm.entities.sdp import BeamMapping
from ska_oso_scripting import oda_helper
from ska_oso_scripting.functions.devicecontrol.resource_control import get_request_json
from ska_oso_scripting.objects import SubArray, Telescope
from ska_tmc_cdm.messages.central_node.assign_resources import AssignResourcesRequest
from ska_tmc_cdm.messages.central_node.sdp import Channel
from ska_tmc_cdm.messages.subarray_node.configure import ConfigureRequest
from ska_tmc_cdm.messages.subarray_node.configure.core import ReceiverBand

from ska_mid_jupyter_notebooks.cluster.cluster import Environment, TangoDeployment
from ska_mid_jupyter_notebooks.dish.dish import TangoDishDeployment
from ska_mid_jupyter_notebooks.helpers.path import project_root
from ska_mid_jupyter_notebooks.obsconfig.config import ObservationSB
from ska_mid_jupyter_notebooks.obsconfig.target_spec import TargetSpec, get_default_target_specs_sb
from ska_mid_jupyter_notebooks.sut.rendering import TelescopeMononitorPlot
from ska_mid_jupyter_notebooks.sut.state import TelescopeDeviceModel, get_telescope_state
from ska_mid_jupyter_notebooks.sut.sut import TangoSUTDeployment, disable_qa
from ska_mid_jupyter_notebooks.test_equipment.rendering import get_test_equipment_monitor_plot
from ska_mid_jupyter_notebooks.test_equipment.state import get_equipment_model
from ska_mid_jupyter_notebooks.test_equipment.test_equipment import TangoTestEquipment


In [5]:
# OPTIONAL - this can be skipped if the libraries are not available
import kubernetes
import ipywidgets as widgets
from IPython.display import display, JSON

ns_list = []
kubernetes.config.load_kube_config()
k8s_client = kubernetes.client.CoreV1Api()
try:
    namespaces = k8s_client.list_namespace(_request_timeout=(1, 5))
    for namespace in namespaces.items:
        ns_name = namespace.metadata.name
        ns_list.append(ns_name)
except Exception:
    pass

ns_valid_w = widgets.Valid(
    value=len(ns_list) > 0,
    description="Namespaces",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(ns_valid_w)

if len(ns_list) > 0:
    ns_w = widgets.Dropdown(
        options=ns_list,
        description='SUT ns',
        disabled=False,
        layout=widgets.Layout(width='50%')
    )
    display(ns_w)

Valid(value=True, description='Namespaces', layout=Layout(width='60%'), style=DescriptionStyle(description_wid…

Dropdown(description='SUT ns', layout=Layout(width='50%'), options=('binderhub', 'calico-apiserver', 'calico-o…

### 1.2 Setup Global Variables and Configuration
##### Defining the configuration of the notebook, like git branch and logging leves

In [6]:
try:
    SUT_NAMESPACE = ns_w.value
    print(f"Using SUT namespace {SUT_NAMESPACE}")
    branch_name = SUT_NAMESPACE.replace("ci-ska-mid-itf-", "")
except Exception:
    branch_name = "at-2141-human-read-nb"

Using SUT namespace ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks


In [7]:
debug_mode = False  # This setting enables printing of diagnostics
enable_logging = False  # This enables logging and sets the global log_level to debug
dishlmc_enabled = True  # Set this to true if you have a dish LMC deployment
executon_environment = Environment.CI

# Set this if you are using an on-demand deployment (i.e. Environment.CI)
# branch_name = "at-2141-human-read-nb"  

if enable_logging:
    # ska_ser_logging.configure_logging(logging.DEBUG)
    ska_ser_logging.configure_logging(logging.INFO)
test_equipment = TangoTestEquipment()
print(f"Test Equipment Configured")
# namespace_override parameter can be used to override auto-configured SUT namespace
sut_namespace_override = ""
subarray_count = 1
subarray_id = 1
sut = TangoSUTDeployment(
    branch_name,
    executon_environment,
    namespace_override=sut_namespace_override,
    subarray_index=subarray_id,
)
print(f"SUT configured")
dish_ids = ["001", "036"]
# namespace_override parameter can be used to override auto-configured dish namespace
dish_namespace_overrides = ["", ""]
dish_deployments: List[TangoDishDeployment] = []
if dishlmc_enabled:
    for i, d in enumerate(dish_ids):
        dish = TangoDishDeployment(
            f"ska{d}",
            branch_name=branch_name,
            environment=executon_environment,
            namespace_override=dish_namespace_overrides[i],
        )
        print(f"Dish {d} configured")
        dish_deployments.append(dish)

timestr = time.strftime("%Y%m%d-%H%M")
notebook_output_dir = pathlib.Path(
    project_root(), f"notebook-execution-data/configure_scan_for_commissioning/execution-{timestr}"
)
os.makedirs(notebook_output_dir, exist_ok=True)
# we disable qa as it is not been properly verified
disable_qa()

Test Equipment Configured
SUT configured
Dish 001 configured
Dish 036 configured


### 1.3 Test Connections to Namespaces
##### Runs smoke tests on all relevant namespaces of the system under test

In [8]:
sut.smoke_test()
test_equipment.smoke_test()
for dish_deployment in dish_deployments:
    dish_deployment.smoke_test()
print("Done")

CIA PingResponse (ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks): {"result":"ok","time":"2024-07-18T10:59:07.991027"}
CIA PingResponse (test-equipment): {"result":"ok","time":"2024-07-18T10:59:08.447528"}
mid-itf/siggen/1 is reachable
mid-itf/progattenuator/1 is reachable
mid-itf/spectana/1 is reachable
mid-itf/skysimctl/4 is reachable
CIA PingResponse (ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks): {"result":"ok","time":"2024-07-18T10:59:09.900942"}
CIA PingResponse (ci-dish-lmc-ska036-at-2146-consolidate-jupyter-notebooks): {"result":"ok","time":"2024-07-18T10:59:12.996432"}
Done


### 1.4 Export System Configuration
##### Prints the helm chart versions of all telescope components in json format

In [9]:
deployment: TangoDeployment
for deployment in [sut, test_equipment, *dish_deployments]:
    try:
        deployment.export_chart_configuration(output_dir=notebook_output_dir)
        print(notebook_output_dir)
    except Exception as err:
        print(f"ERROR: {str(err)}")
print("Done")

Exporting configuration using http://config-inspector.ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks.svc.miditf.internal.skao.int:8765
ReleaseResponse (ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks): {
    "chart": "ska-mid-itf-sut",
    "version": "23.3.0",
    "sub_charts": [
        {
            "chart": "ska-tango-base",
            "version": "0.4.10"
        },
        {
            "chart": "ska-tango-util",
            "version": "0.4.11"
        },
        {
            "chart": "ska-tmc-mid",
            "version": "0.21.0"
        },
        {
            "chart": "ska-csp-lmc-mid",
            "version": "0.20.1"
        },
        {
            "chart": "ska-mid-cbf-mcs",
            "version": "0.15.2"
        },
        {
            "chart": "ska-sdp",
            "version": "0.21.0"
        },
        {
            "chart": "ska-ser-config-inspector",
            "version": "0.2.3"
        },
        {
            "chart": "ska-tango-taranta",
           

## 2 Telescope Setup

#### 2.1 Configure Telescope Monitoring
##### Setup how the telescope will subscribe to events

In [10]:
# setup monitoring
# use telescope state object for state monitoring
device_model = TelescopeDeviceModel(dish_ids, subarray_count)
telescope_state = get_telescope_state(device_model, sut)
# use monitor plot as a dashboard
telescope_monitor_plot = TelescopeMononitorPlot(plot_width=900, plot_height=200)
# set up events to monitor
telescope_state.subscribe_to_on_off(telescope_monitor_plot.observe_telescope_on_off)
telescope_state.subscribe_to_subarray_resource_state(
    telescope_monitor_plot.observe_subarray_resources_state
)
telescope_state.subscribe_to_subarray_configurational_state(
    telescope_monitor_plot.observe_subarray_configuration_state
)
telescope_state.subscribe_to_subarray_scanning_state(
    telescope_monitor_plot.observe_subarray_scanning_state
)
output_notebook()

Loading BokehJS ...

#### 2.2 Open the inline dashboard
Start the simple inline dashboard showing current state of the Telescope and resource assignment and configuration status.

In [11]:
telescope_monitor_plot.show()
telescope_state.activate()
telescope_state.wait_til_ready(2)
print("Done")

Done


### 2.3 Print System Diagnostics

#### 2.3.1 Print TMC Diagnostics

In [12]:
try:
    sut.print_tmc_diagnostics()
except Exception as err:
    print(f"ERROR: {str(err)}")
print("Done")

TMC Central Node state: ON
TMC Central Node adminMode: AdminMode.OFFLINE
TMC Central Node healthState: HealthState.OK
TMC Central Node telescopeHealthState: HealthState.UNKNOWN
TMC Central Node isDishVccConfig: False
TMC Central Node dishvccvalidationstatus: {"dish": "ALL DISH OK", "ska_mid/tm_leaf_node/csp_master": "TMC and CSP Master Dish Vcc Version is Same"}
TMC Subarray Node state: ON
TMC Subarray adminMode: AdminMode.OFFLINE
TMC Subarray Node obsState: ObsState.EMPTY
Done


#### 2.3.2 Print CSP-LMC Diagnostics

In [13]:
try:
    sut.print_csp_diagnostics()
except Exception as err:
    print(f"ERROR: {str(err)}")
print("Done")

CSP-LMC Controller adminMode: AdminMode.ONLINE
CSP-LMC Controller State: OFF
CSP-LMC Controller dishVccConfig: {"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "dish_parameters": {"SKA001": {"vcc": 1, "k": 1}, "SKA036": {"vcc": 2, "k": 1}, "SKA063": {"vcc": 3, "k": 1}, "SKA100": {"vcc": 4, "k": 1}}}
CSP-LMC Controller CBFSimulationMode: False
CSP-LMC Subarray adminMode: AdminMode.ONLINE
CSP-LMC Subarray State: OFF
CSP-LMC Subarray obsState: ObsState.EMPTY
CSP-LMC Subarray dishVccConfig: {"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "dish_parameters": {"SKA001": {"vcc": 1, "k": 1}, "SKA036": {"vcc": 2, "k": 1}, "SKA063": {"vcc": 3, "k": 1}, "SKA100": {"vcc": 4, "k": 1}}}
Done


#### 2.3.3 Print CBF Diagnostics

In [14]:
try:
    sut.print_cbf_diagnostics()
except Exception as err:
    print(f"ERROR: {str(err)}")
print("Done")

CBF Controller adminMode: AdminMode.ONLINE
CBF Controller State: OFF
CBF Subarray adminMode: AdminMode.ONLINE
CBF Subarray State: OFF
CBF Subarray obsState: ObsState.EMPTY
Done


#### 2.3.4 Print SDP Diagnostics

In [15]:
try:
    sut.print_sdp_diagnostics()
except Exception as err:
    print(f"ERROR: {str(err)}")
print("Done")

SDP Controller state: ON
SDP Controller adminMode: AdminMode.ONLINE
SDP Subarray state: ON
SDP Subarray adminMode: AdminMode.ONLINE
SDP Subarray obsState: ObsState.EMPTY
Done


#### 2.3.5 Print Dish-LMC Diagnostics

In [16]:
for dish_deployment in dish_deployments:
    # dish_deployment.print_diagnostics()
    print(f"Dish {dish_deployment.dish_id} - {dish_deployment.namespace}: Diagnostics")
    dish_id = dish_deployment.dish_id
    dm = dish_deployment.dish_manager
    print(f"{dish_id}: PowerState: {str(dm.power_state)}")
    print(f"{dish_id}: HealthState: {str(dm.health_state)}")
    print(f"{dish_id}: PointingState: {str(dm.pointing_state)}")
    print(f"{dish_id}: K-Value: {dm.kValue}")
    print(f"{dish_id}: Capturing: {dm.capturing}")
    print(f"{dish_id}: SimulationMode: {dm.simulationMode}")
    spfc = dish_deployment.spfc_simulator
    print(f"{dish_id}: SPFC OperatingMode: {str(spfc.operating_mode)}")
    spfrx = dish_deployment.spfrx
    print(f"{dish_id}: SPFRx OperatingMode: {str(spfrx.operating_mode)}")
    ds_manager = dish_deployment.ds_manager
    print(f"{dish_id}: DS Manager OperatingMode: {str(ds_manager.operating_mode)}")
    print(f"{dish_id}: DS Manager IndexerPosition: {ds_manager.indexerPosition}")
    print()
print("Done")

Dish ska001 - ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks: Diagnostics
ska001: PowerState: PowerState.LOW
ska001: HealthState: HealthState.UNKNOWN
ska001: PointingState: PointingState.READY
ska001: K-Value: 1
ska001: Capturing: False
ska001: SimulationMode: 0
ska001: SPFC OperatingMode: SPFOperatingMode.OPERATE
ska001: SPFRx OperatingMode: SPFRxOperatingMode.STANDBY
ska001: DS Manager OperatingMode: DSOperatingMode.STANDBY_FP
ska001: DS Manager IndexerPosition: 2

Dish ska036 - ci-dish-lmc-ska036-at-2146-consolidate-jupyter-notebooks: Diagnostics
ska036: PowerState: PowerState.LOW
ska036: HealthState: HealthState.UNKNOWN
ska036: PointingState: PointingState.READY
ska036: K-Value: 1
ska036: Capturing: False
ska036: SimulationMode: 0
ska036: SPFC OperatingMode: SPFOperatingMode.STANDBY_LP
ska036: SPFRx OperatingMode: SPFRxOperatingMode.STANDBY
ska036: DS Manager OperatingMode: DSOperatingMode.STANDBY_LP
ska036: DS Manager IndexerPosition: 2

Done


#### 2.3.6 Print Full System Diagnostics

In [17]:
print("SUT: Diagnostics")
# sut.print_full_diagnostics()

for chart in sut.release.sub_charts:
    devices = sut.chart_devices(chart.chart)
    for device in devices:
        print(f"{chart.chart}: {device.name} : {device.deployment_status}")

for dish_deployment in dish_deployments:
    # dish_deployment.print_full_diagnostics()
    print(f"\nDish {dish_deployment.dish_id}: Diagnostics")
    for chart in dish_deployment.release.sub_charts:
        devices = dish_deployment.chart_devices(chart.chart)
        for device in devices:
            print(f"{chart.chart}: {device.name} : {device.deployment_status}")
print("Done")

SUT: Diagnostics
ska-tango-base: sys/tg_test/1 : Running
ska-tmc-mid: ska_mid/tm_leaf_node/csp_subarray01 : Unknown
ska-tmc-mid: ska_mid/tm_leaf_node/d0001 : Unknown
ska-tmc-mid: ska_mid/tm_central/central_node : Unknown
ska-tmc-mid: ska_mid/tm_leaf_node/csp_master : Unknown
ska-tmc-mid: ska_mid/tm_leaf_node/sdp_master : Unknown
ska-tmc-mid: ska_mid/tm_leaf_node/sdp_subarray02 : Unknown
ska-tmc-mid: ska_mid/tm_subarray_node/2 : Unknown
ska-tmc-mid: ska_mid/tm_subarray_node/1 : Unknown
ska-tmc-mid: ska_mid/tm_leaf_node/sdp_subarray01 : Unknown
ska-tmc-mid: ska_mid/tm_leaf_node/csp_subarray02 : Unknown
ska-tmc-mid: ska_mid/tm_leaf_node/d0036 : Unknown
ska-csp-lmc-mid: mid-csp/capability-vcc/0 : Running
ska-csp-lmc-mid: mid-csp/subarray/01 : Running
ska-csp-lmc-mid: mid-csp/subarray/03 : Running
ska-csp-lmc-mid: mid-csp/subarray/02 : Running
ska-csp-lmc-mid: sys/tg_test/1 : Unknown
ska-csp-lmc-mid: mid-csp/capability-fsp/0 : Running
ska-csp-lmc-mid: mid-csp/control/0 : Running
ska-mid-cbf

### 2.4 Setup ODA

In [18]:
os.environ["ODA_URI"] = (
    "http://ingress-nginx-controller-lb-default.ingress-nginx.svc.miditf.internal.skao.int/ska-db-oda/api/v1/"
)
eb_id = oda_helper.create_eb()
print(f"Execution Block ID: {eb_id}")
print("Done")

TypeError: create_eb() missing 1 required positional argument: 'telescope'

Thu Jul 18 13:01:08 2024: Tango::ZmqEventConsumer::push_heartbeat_event() timeout on channel monitor of tango://tango-databaseds.ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks.svc.miditf.internal.skao.int:10000/dserver/central_node_mid/01


### 2.5 Initialise Telescope and Subarray
Create Subarray and Telescope instances.

In [19]:
sub = SubArray(subarray_id)
tel = Telescope()
print("Done")

Done


### 2.6 Load VCC Configuration in TMC

In [20]:
# This should only be executed for a fresh deployment (i.e. Telescope is OFF.
# If you have restarted the subarray, you should not run this command
sut.load_dish_vcc_config()
print("Done")

CSP Controller: adminMode=0; State=OFF
CSP Controller: adminMode=0; State=OFF after 0s.
CSP Controller: adminMode=0; State=OFF
TMC Central Node isDishVccConfigSet=False after 0s
TMC Central Node isDishVccConfigSet=False after 1s
TMC Central Node isDishVccConfigSet=False after 3s
TMC Central Node isDishVccConfigSet=False after 7s
TMC Central Node isDishVccConfigSet=False after 15s
TMC Central Node isDishVccConfigSet=False after 31s


Thu Jul 18 13:03:41 2024: Tango::ZmqEventConsumer::push_heartbeat_event() timeout on channel monitor of tango://tango-databaseds.ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks.svc.miditf.internal.skao.int:10000/dserver/central_node_mid/01


TMC Central Node isDishVccConfigSet=False after 63s
TMC Central Node isDishVccConfigSet=False after 127s


KeyboardInterrupt: 

### 2.7 Turn telescope ON

In [23]:
# set to ON only if OFF
# If you have restarted the subarray, you should not run this command (Telescope is already ON)
# dish_lmc mode must be in LP_standby and before trying to turn the telescope ON
# Takes about 1m20s
print(f"Telescope is {telescope_monitor_plot.on_off_state}")
if telescope_monitor_plot.on_off_state == "OFF":  # e.g. purple
    tel.on()
else:
    assert (
        telescope_monitor_plot.on_off_state == "ON"
    ), f"Cant continue with telescope in {telescope_monitor_plot.on_off_state}"
print(f"Telescope is {telescope_monitor_plot.on_off_state}")

Telescope is OFF
1|2024-07-18T11:05:22.190Z|ERROR|MainThread|wrapper|oda_helper.py#245||The ODA_URL not set, meaning Execution Block updates will not be sent to the ODA. Please set this variable to the URL for an instance of the ODA, eg ODA_URL=http://k8s.stfc.skao.int/staging-ska-db-oda/
1|2024-07-18T11:05:22.191Z|INFO|MainThread|_call_and_wait_for_transition|common.py#147||Using pub/sub to track telescopeState of ska_mid/tm_central/central_node
1|2024-07-18T11:05:22.191Z|INFO|MainThread|execute|tango_executor.py#243||Executing command: <Command('ska_mid/tm_central/central_node', 'TelescopeOn', )>


DevFailed: DevFailed[
DevError[
    desc = ska_tmc_common.exceptions.CommandNotAllowed: Dish Vcc Config not Set. Please set using LoadDishCfg command. Current Telescope State is :ON
           
  origin = Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tango/device_server.py", line 85, in wrapper
    return get_worker().execute(fn, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tango/green.py", line 101, in execute
    return fn(*args, **kwargs)
  File "/app/src/ska_tmc_centralnode/central_node.py", line 299, in is_TelescopeOn_allowed
    return self.component_manager.is_command_allowed("TelescopeOn")
  File "/app/src/ska_tmc_centralnode/manager/component_manager_mid.py", line 508, in is_command_allowed
    raise CommandNotAllowed(
ska_tmc_common.exceptions.CommandNotAllowed: Dish Vcc Config not Set. Please set using LoadDishCfg command. Current Telescope State is :ON

  reason = PyDs_PythonError
severity = ERR]

DevError[
    desc =

Thu Jul 18 13:06:15 2024: Tango::ZmqEventConsumer::push_heartbeat_event() timeout on channel monitor of tango://tango-databaseds.ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks.svc.miditf.internal.skao.int:10000/dserver/csp_master_leaf_node/01
Thu Jul 18 13:18:23 2024: Tango::ZmqEventConsumer::push_heartbeat_event() timeout on channel monitor of tango://tango-databaseds.ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks.svc.miditf.internal.skao.int:10000/dserver/central_node_mid/01
Thu Jul 18 13:20:56 2024: Tango::ZmqEventConsumer::push_heartbeat_event() timeout on channel monitor of tango://tango-databaseds.ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks.svc.miditf.internal.skao.int:10000/dserver/csp_master_leaf_node/01
Thu Jul 18 13:23:29 2024: Tango::ZmqEventConsumer::push_heartbeat_event() timeout on channel monitor of tango://tango-databaseds.ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks.svc.miditf.internal.skao.int:10000/dserver/central_node_mid/01
Thu Jul 18 1

In [22]:
print(f"Telescope is {telescope_monitor_plot.on_off_state}")

Telescope is OFF


## 3. Observation Definition

#### 3.1 Create the high level observation specifications in terms of target specs

Note :- Users may currently modify the values by replacing the example values as given for each field within Target specification section.

In [ ]:
dish_ids = [d.dish_id.upper() for d in dish_deployments]
default_target_specs = get_default_target_specs_sb(dish_ids)
observation = ObservationSB(target_specs=default_target_specs)

target_specs = {
    "flux calibrator": TargetSpec(
        target_sb_detail={
            "co_ordinate_type": "Equatorial",
            "ra": "19:24:51.05 degrees",
            "dec": "-29:14:30.12 degrees",
            "reference_frame": "ICRS",
            "unit": ("hourangle", "deg"),
            "pointing_pattern_type": {
                "single_pointing_parameters": SinglePointParameters(
                    offset_x_arcsec=0.0, offset_y_arcsec=0.0
                ),
                "raster_parameters": RasterParameters(
                    row_length_arcsec=0.0,
                    row_offset_arcsec=0.0,
                    n_rows=1,
                    pa=0.0,
                    unidirectional=False,
                ),
                "star_raster_parameters": StarRasterParameters(
                    row_length_arcsec=0.0,
                    n_rows=1,
                    row_offset_angle=0.0,
                    unidirectional=False,
                ),
                "five_point_parameters": FivePointParameters(offset_arcsec=0.0),
                "cross_scan_parameters": CrossScanParameters(offset_arcsec=0.0),
                "active_pointing_pattern_type": "single_pointing_parameters",
            },
        },
        scan_type="flux calibrator",
        band=ReceiverBand.BAND_2,
        channelisation="vis_channels9",
        polarisation="all",
        processing="test-receive-addresses",
        dish_ids=dish_ids,
        target=None,
    ),
    "M87": TargetSpec(
        target_sb_detail={
            "co_ordinate_type": "Equatorial",
            "ra": "19:24:51.05 degrees",
            "dec": "-29:14:30.12 degrees",
            "reference_frame": "ICRS",
            "unit": ("hourangle", "deg"),
            "pointing_pattern_type": {
                "single_pointing_parameters": SinglePointParameters(
                    offset_x_arcsec=0.0, offset_y_arcsec=0.0
                ),
                "raster_parameters": RasterParameters(
                    row_length_arcsec=0.0,
                    row_offset_arcsec=0.0,
                    n_rows=1,
                    pa=0.0,
                    unidirectional=False,
                ),
                "star_raster_parameters": StarRasterParameters(
                    row_length_arcsec=0.0,
                    n_rows=1,
                    row_offset_angle=0.0,
                    unidirectional=False,
                ),
                "five_point_parameters": FivePointParameters(offset_arcsec=0.0),
                "cross_scan_parameters": CrossScanParameters(offset_arcsec=0.0),
                "active_pointing_pattern_type": "single_pointing_parameters",
            },
        },
        scan_type="M87",
        band=ReceiverBand.BAND_2,
        channelisation="vis_channels10",
        polarisation="all",
        processing="test-receive-addresses",
        dish_ids=dish_ids,
        target=None,
    ),
}


channel_configuration = [
    Channel(
        spectral_window_id="fsp_1_channels",
        count=14880,
        start=0,
        stride=2,
        freq_min=0.35e9,
        freq_max=0.368e9,
        link_map=[[0, 0], [200, 1], [744, 2], [944, 3]],
    )
]

for key, value in target_specs.items():
    observation.add_channel_configuration(value.channelisation, channel_configuration)

observation.add_target_specs(target_specs)

for target_id, target in target_specs.items():
    observation.add_scan_type_configuration(
        config_name=target_id,
        beams={"vis0": BeamMapping(beam_id="vis0", field_id="M83")},
        derive_from=".default",
    )
scan_def_id = "flux calibrator"
observation.add_scan_sequence([scan_def_id])
print("Done")

#### 3.2 Mid configuration schema input used by observing commands

[Configuration Schemas-OET→TMC(Mid)](https://developer.skao.int/projects/ska-telmodel/en/latest/)

In [ ]:
telescope_monitor_plot.show()

#### 3.3 Create Scheduling Block Definition(SBD) Instance and save it into the ODA

In [ ]:
observation.eb_id = eb_id
try:
    pdm_allocation = observation.generate_pdm_object_for_sbd_save(target_specs)
    sbd = oda_helper.save(pdm_allocation)
    sbd_id = sbd.sbd_id
    pdm_allocation.sbd_id = sbd_id
    print(f"Saved Scheduling Block Definition Instance in ODA: SBD_ID={sbd_id}")
except Exception as err:
    pdm_allocation = None
    print(f"ERROR: {str(err)}")

### 3.4 Assign Resources
Assign the requested resources to a Subarray

In [ ]:
if pdm_allocation is not None:
    assign_request = observation.generate_allocate_config_sb(pdm_allocation).as_object

    if debug_mode:
        request_json = get_request_json(assign_request, AssignResourcesRequest, True)
        print("AssignResourcesRequest:", json.dumps(json.loads(request_json), indent=2))

    sub.assign_from_cdm(assign_request, timeout=120)
else:
    print("ERROR: PDM not allocated")
print("Done")

### 3.5  Show telescope status

In [ ]:
telescope_monitor_plot.show()

### 3.7 Configure Scan
Configure the telescope  on first target in sequence - may be modified to configure and run multiple targets at a later time.

In [ ]:
if pdm_allocation is not None:
    configure_object = observation.generate_scan_config_sb(
        pdm_observation_request=pdm_allocation,
        scan_definition_id=scan_def_id,
        scan_duration=10.0,
    ).as_object

    if debug_mode:
        cfg_json = get_request_json(configure_object, ConfigureRequest)
        print(f"ConfigureRequest={cfg_json}")

    try:
        sub.configure_from_cdm(configure_object, timeout=120)
    except tango.DevFailed as terr:
        print(f"ERROR: {terr.args[0].desc.strip()}")
    except Exception as evt_err:
        print("ERROR: {str(evt_err)}")
    time.sleep(2)
else:
    print("ERROR: PDM not allocated")
print("Done")

### 3.8 Post Observation teardown
If the observation executed successfully, you can use the following commands to reset the telescope.

#### 3.8.1 Clear scan configuration 

In [ ]:
try:
    sub.end()
except tango.DevFailed as derr:
    err_msg = derr.args[0].desc.strip()
    print(f"ERROR: {err_msg}")
telescope_monitor_plot.show()

#### 3.8.2 Release Subarray resources

In [ ]:
try:
    sub.release()
except tango.DevFailed as derr:
    err_msg = derr.args[0].desc.strip()
    print(f"ERROR: {err_msg}")
telescope_monitor_plot.show()

## 4. Turn off or reset the telescope

## 4.1 Reset the telescope

### 4.1.1 Reset the Subarray (On Failure)
Set booleans to True to reset the system after a failed execution.

In [ ]:
try:
    sub.abort()
    time.sleep(3)
    sub.restart()
except tango.DevFailed as derr:
    err_msg = derr.args[0].desc.strip()
    print(f"ERROR: {err_msg}")
print("Done")

## 4.2 Turn off telescope

### 4.2.1 Send off command

In [ ]:
print(f"Telescope is {telescope_monitor_plot.on_off_state}")
if telescope_monitor_plot.on_off_state != "OFF":  # e.g. purple
    tel.off()
    sleep(3)
    print(f"Telescope is {telescope_monitor_plot.on_off_state}")
else:
    print("ERROR: telescope is already off")

In [ ]:
print(f"Telescope is {telescope_monitor_plot.on_off_state}")

## 5. Configure System for Scan Commissioning Test

### Copied from Jama 2024-04-15

### 4.1 Bring the sub-systems to STANDBY mode using Telescope ON script or command from TMC as applicable.

Running steps 1-3.3 should get the telescope into a standby state which can be checked by running 2.3

### 4.2 Inspect the TMC API or dashboard to view the indication of connectivity status and mode of operation of the sub-systems.

### 4.3 Change the telescope state to operational
Running steps 3.4. - 3.5 should get the telescope to operational mode which can be checked by running steps 2.3

## 5. Hyperlinks and SUT config


### 5.1 Gitlab pipeline used for the test: 
https://gitlab.com/ska-telescope/ska-mid-itf/-/pipelines/1361824989

### 5.2 SUT config:

{
  "chart": "ska-mid-itf",
  "version": "23.2.0",
  "sub_charts": [
    {
      "chart": "ska-tango-base",
      "version": "0.4.10"
    },
    {
      "chart": "ska-tango-util",
      "version": "0.4.11"
    },
    {
      "chart": "ska-tmc-mid",
      "version": "0.20.0"
    },
    {
      "chart": "ska-csp-lmc-mid",
      "version": "0.20.1"
    },
    {
      "chart": "ska-mid-cbf-mcs",
      "version": "0.15.1"
    },
    {
      "chart": "ska-sdp",
      "version": "0.21.0"
    },
    {
      "chart": "ska-ser-config-inspector",
      "version": "0.2.3"
    },
    {
      "chart": "ska-tango-taranta",
      "version": "2.10.2"
    },
    {
      "chart": "ska-tango-tangogql",
      "version": "1.4.3"
    },
    {
      "chart": "ska-tango-alarmhandler",
      "version": "0.4.0"
    },
    {
      "chart": "ska-tango-archiver",
      "version": "2.7.1"
    }
  ]
}